# AIM Hackathon: Sample code
19.10.2024

In [6]:
import os
import requests
import PyPDF2
import tiktoken
import pandas as pd
import pickle
from dotenv import load_dotenv
from tqdm import tqdm

from langchain_openai import ChatOpenAI
from langchain_core.documents import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai.embeddings.base import OpenAIEmbeddings

from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.vectorstores import FAISS

from Cryptodome.Cipher import AES

from typing import Optional, List, Union
from pydantic import BaseModel
from openai import OpenAI
import warnings

# Suppress only LangChainDeprecationWarning
warnings.filterwarnings("ignore", category=DeprecationWarning, module="langchain")
warnings.filterwarnings("ignore", message="Importing FAISS from langchain.vectorstores is deprecated. Please replace deprecated imports")
# load openai key
if not load_dotenv():
    raise Exception('Error loading .env file. Make sure to place a valid OPEN_AI_KEY in the .env file.')

In [7]:
REPORTS_SAVE_PATH = 'data/sample_reports'
DB_PATH = "data/db/sample.db"

# See https://openai.com/api/pricing/
MODEL = "gpt-4o"

In [8]:
df = pd.read_json('data/reports.json')
df

,company_name,year,dataset,pdf_url
0,Walmart,2023,handcrafted,https://corporate.walmart.com/content/dam/corp...
1,Walmart,2021,handcrafted,https://corporate.walmart.com/content/dam/corp...
2,Walmart,2019,handcrafted,https://corporate.walmart.com/content/dam/corp...
3,Amazon,2023,handcrafted,https://sustainability.aboutamazon.com/content...
4,Amazon,2021,handcrafted,https://sustainability.aboutamazon.com/content...
...,...,...,...,...
141,tarkett,2020,scraped,https://www.tarkett.com/sites/default/files/20...
142,trivium-packaging,2021,scraped,https://www.triviumpackaging.com/media/13fl4q3...
143,trivium-packaging,2020,scraped,https://triviumpackaging.com/sustainability/re...
144,trust,2023,scraped,https://dezlwerqy1h00.cloudfront.net/images/co...


## Download some reports

In [9]:
# EXAMPLE: select apple reports
df_sample = df[df['dataset']=='handcrafted']

In [10]:
# Storing the encryption keys for further decryption
enc_keys = []

# download Apple reports to save_dir
def download_files(df: pd.DataFrame, save_dir: str):
    os.makedirs(save_dir, exist_ok=True)
    for url in df['pdf_url']:
        pdf_filename = os.path.basename(url)
        # Checking if the file is encrypted
        if('?' in pdf_filename):
            # Saving the password for decryption
            enc_keys.append(pdf_filename)
            # Removing question mark
            pdf_filename = pdf_filename.split('?')[0]
            
        response = requests.get(url)
        with open(os.path.join(save_dir, pdf_filename), 'wb') as file:
            file.write(response.content)
    print(f"Success.")

In [11]:
download_files(df_sample, REPORTS_SAVE_PATH)

Success.


## Create simple vector database

In [12]:
def get_password(f):
    for tmp in enc_keys:
        if(f == tmp.split()[0]):
            return tmp

def get_documents_from_path(files_path: str) -> [Document]:
    documents = []
    
    for file in os.listdir(files_path):
        _, file_extension = os.path.splitext(file)
        text = ""
        
        if file_extension == ".pdf":
            with open(os.path.join(files_path, file), 'rb') as f:
                reader = PyPDF2.PdfReader(f, strict=False)
                
                if reader.is_encrypted:
                    try:
                        # Try to decrypt with the provided password (or an empty string if no password is given)
                        pdf_password = get_password(file)
                        
                        if pdf_password:
                            success = reader.decrypt(pdf_password)
                        else:
                            success = reader.decrypt("")

                        if success == 0:
                            print(f"Failed to decrypt {file}: Invalid password.")
                            continue  # Skip file if decryption fails
                        else:
                            print(f"Decrypted {file} successfully.")
                    except Exception as e:
                        print(f"Failed to decrypt {file}: {e}")
                        continue  # Skip file if decryption fails
                for page in reader.pages:
                    text += page.extract_text() + "\n"
                
            if text:
                documents.append(Document(page_content=text, metadata={"source": file}))
            else:
                print(f"WARNING: No text extracted from {file}")
        else:
            # TODO: can add support for other file types here
            raise Exception(f"Unsupported file extension: {file_extension}")
    
    return documents

In [13]:
documents = get_documents_from_path(REPORTS_SAVE_PATH)

Decrypted 2023_Volkswagen_Group_Sustainability_Report.pdf successfully.
Decrypted bp-sustainability-report-2021.pdf successfully.
Decrypted bp-sustainability-report-2023.pdf successfully.
Decrypted Nonfinancial_Report_2021_en.pdf successfully.


In [14]:
# TODO could also just provide a dummy retriever to not spoil too much
class DummyRetriever:
    def __init__(self, texts):
        self.texts = texts
        
    def dummy_retriever(self, query):
        import random
        return random.sample(self.texts, k=3)

In [15]:
# Create database
text_splitter = RecursiveCharacterTextSplitter(chunk_size=3000, chunk_overlap=300, separators=["\n\n", "\n"])

# split documents and create vector database
texts = text_splitter.split_documents(documents)
embeddings = OpenAIEmbeddings()  # https://platform.openai.com/docs/guides/embeddings/embedding-models
db = FAISS.from_documents(texts, embeddings)

# count build embedding token number
tokenizer = tiktoken.get_encoding("cl100k_base")
build_token_count = sum([len(tokenizer.encode(doc.page_content)) for doc in texts])
print(f"Token count: {build_token_count}")

Token count: 1286489


In [16]:
# Store the database
with open(DB_PATH, "wb") as f:
    pickle.dump(db.serialize_to_bytes(), f)

## Create simple RAG

In [17]:
# Load the database
DB_PATH = "data/db/sample.db"

with open(DB_PATH, "rb") as f:
    db_bytes = pickle.load(f)
    db = FAISS.deserialize_from_bytes(db_bytes, OpenAIEmbeddings(), allow_dangerous_deserialization=True)

In [18]:
client = OpenAI()

retriever=db.as_retriever()

class Answer(BaseModel):
    value: Optional[List[Union[float, int]]]
    unit: str
    chain_of_thought: str

def retrieve_context(question):
    context_docs = retriever.get_relevant_documents(question)
    context = '\n'.join([doc.page_content for doc in context_docs])
    return context

def construct_messages(context, question):
    system_prompt = (
        "You are an expert assistant. Use only the following retrieved context to answer the question accurately and concisely. "
        "Provide your answer as a number followed by its unit, without any additional text or explanation. "
        "Before giving the final answer, include your chain-of-thought reasoning prefixed with 'Chain of Thought:'. "
        "If nothing is mentioned in the context, say 'I don't know'."
    )
    
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": f"Context:\n{context}\n\nQuestion:\n{question}"}
    ]
    return messages

def get_response_from_openai(question, verbose=True):
    context = retrieve_context(question)
    messages = construct_messages(context, question)
    completion = client.beta.chat.completions.parse(
        model=MODEL,
        messages=messages,
        response_format=Answer
        )
    answer = completion.choices[0].message.parsed

    if verbose:
        if answer.value is not None:
            print(f"Value: {answer.value}")
            print(f"Unit: {answer.unit}")
            print(f"Chain of Thought: {answer.chain_of_thought}")
        else:
            print("Cannot answer.")
            print(f"Chain of Thought: {answer.chain_of_thought}")

    return answer


In [19]:
esg_metrics = [
    'Carbon Emissions',
    'Energy Consumption',
    'Water Usage',
    'Waste Generation',
    'Renewable Energy Usage',
    'Greenhouse Gas Emissions Intensity',
    'Biodiversity Impact',
    'Air Pollutant Emissions',
    'Environmental Management System',
    'Supply Chain Environmental Impact',
    'Employee Turnover Rate',
    'Gender Diversity Ratio',
    'Employee Health and Safety Incidents',
    'Labor Practices',
    'Human Rights Compliance',
    'Community Engagement Initiatives',
    'Customer Satisfaction Score',
    'Data Privacy Breaches',
    'Product Safety Incidents',
    'Board Diversity',
    'Executive Compensation Ratio',
    'Shareholder Rights',
    'Anti-Corruption Policies',
    'Regulatory Compliance',
    'Ethical Supply Chain Management',
    'Stakeholder Engagement',
    'Risk Management Strategies',
    'Transparency and Disclosure Practices',
    'Climate Change Risks and Opportunities',
    'Innovation and R&D Investment'
]


In [20]:
esg_metrics2 = [

    'Renewable Energy Usage',
    'Greenhouse Gas Emissions Intensity',
    'Biodiversity Impact',
    'Air Pollutant Emissions',
    'Environmental Management System',
    'Supply Chain Environmental Impact',
    'Employee Turnover Rate',
    'Gender Diversity Ratio',
    'Employee Health and Safety Incidents',
    'Labor Practices',
    'Human Rights Compliance',
    'Community Engagement Initiatives',
    'Customer Satisfaction Score',
    'Data Privacy Breaches',
    'Product Safety Incidents',
    'Board Diversity',
    'Executive Compensation Ratio',
    'Shareholder Rights',
    'Anti-Corruption Policies',
    'Regulatory Compliance',
    'Ethical Supply Chain Management',
    'Stakeholder Engagement',
    'Risk Management Strategies',
    'Transparency and Disclosure Practices',
    'Climate Change Risks and Opportunities',
    'Innovation and R&D Investment'
]


In [21]:
get_response_from_openai(f"in which years do we have data on co2 emission for Apple?")

C:\Users\bokovacs\AppData\Local\Temp\ipykernel_20768\3602727695.py:11: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  context_docs = retriever.get_relevant_documents(question)


Value: [2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018]
Unit: year
Chain of Thought: Chain of Thought: Analyzing the provided context, data on Apple's CO2 emissions are given explicitly for each year from 2011 to 2018.


Answer(value=[2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018], unit='year', chain_of_thought="Chain of Thought: Analyzing the provided context, data on Apple's CO2 emissions are given explicitly for each year from 2011 to 2018.")

In [22]:
# Initialize the final DataFrame
df_final2 = pd.DataFrame(columns=["company", "year", "metric", "value", "unit"])

# Display the available companies
companies = set(df_sample.company_name)
print(f"The companies are: {', '.join(companies)}")

# Initialize variables
add_more_metrics = True
all_metrics = False

# Prompt the user to enter a metric or 'all'
while True:
    metric_input = input("Please enter a metric you are interested in. If you need some tips, type 'help'. To use all metrics, type 'all'.\nMetric: ").strip()
    if metric_input.lower() == "help":
        print("\nAvailable ESG metrics:")
        print(', '.join(esg_metrics))
        print()
    elif metric_input.lower() == "all":
        all_metrics = True
        metrics_list = esg_metrics2  # Use all metrics
        break
    else:
        metrics_list = [metric_input]
        break

# Main loop to collect data
while add_more_metrics:
    for metric in metrics_list:
        # Iterate over companies and collect data
        for company in companies:
            years_response = get_response_from_openai(f"In which years do we have data on {metric} for {company}?")
            if years_response.value is not None:
                # Clean and split the years
                years_list = years_response.value
                for year in years_list:
                    response = get_response_from_openai(f"In {year}, what is the {metric} for {company}?")
                    if response.value is not None:
                        print(f"{company} ({year}) - {metric}: {response}")
                        value, unit = response.value, response.unit
                        # Create a new row and append it to df_final
                        new_row = {
                            "company": company,
                            "year": year,
                            "metric": metric,
                            "value": value,
                            "unit": unit
                        }
                        df_final2 = pd.concat([df_final2, pd.DataFrame([new_row])], ignore_index=True)

    # If 'all' was selected, no need to ask for more metrics
    if all_metrics:
        add_more_metrics = False
    else:
        # Ask if the user wants to add another metric
        while True:
            continue_input = input("Do you want to add another metric? (yes/no): ").strip().lower()
            if continue_input in ['yes', 'no']:
                break
            else:
                print("Please enter 'yes' or 'no'.\n")

        if continue_input == 'no':
            add_more_metrics = False
        else:
            # Prompt the user to enter the next metric
            while True:
                metric_input = input("Please enter the next metric you are interested in. If you need some tips, type 'help'.\nMetric: ").strip()
                if metric_input.lower() == "help":
                    print("\nAvailable ESG metrics:")
                    print(', '.join(esg_metrics))
                    print()
                else:
                    metrics_list = [metric_input]
                    break


The companies are: H&M, Apple, Google, Volkswagen, Amazon, Saudi Aramco, Walmart, BP
Value: [2016, 2017, 2018, 2019, 2020, 2021, 2022]
Unit: years
Chain of Thought: The data presented for H&M's renewable energy usage is given for the years 2016, 2018, 2019, and 2022. Therefore, these are the years with available data.
Value: [96]
Unit: %
Chain of Thought: The renewable energy usage in H&M's own operations in the past reports is consistently at 96%, as mentioned in the context. While specific figures for 2016 aren't provided, the recurring figure suggests 96% as a likely value for that period as well.
H&M (2016) - Renewable Energy Usage: value=[96] unit='%' chain_of_thought="The renewable energy usage in H&M's own operations in the past reports is consistently at 96%, as mentioned in the context. While specific figures for 2016 aren't provided, the recurring figure suggests 96% as a likely value for that period as well."
Value: [95]
Unit: %
Chain of Thought: The context states the perce

KeyboardInterrupt: 

In [4]:
import pandas as pd
df_final = pd.read_csv("data/df_final.csv")

In [23]:
df_final.metric.unique()

array(['Carbon Emissions', 'Energy Consumption', 'Water Usage',
       'Waste Generation', 'Renewable Energy Usage',
       'Greenhouse Gas Emissions Intensity'], dtype=object)